# Focal Adhesion Analysis
This notebook will analyze the data produced from the 
segmentation macro in FIJI, compute some statistics,
and generate visualizations for those statistics.

### Install dependencies
We need the following libraries for the analysis:
- Pandas
- Scipy
- Matplotlib
- Numpy (a dependency of other libraries, so it is installed implicitly)

In [ ]:
%%capture
# Install required dependencies: Pandas, Scipy, Matplotlib (and Numpy, implicitly)
import sys
!{sys.executable} -m pip install matplotlib pandas scipy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from scipy.stats import linregress
import numpy as np

### Read in the segmentation data
We have two sources of data we want to analyze: the results of the 
"Analyze Particles" plugin in FIJI, which will give us the following
information for each focal adhesion:
- centroid location (x,y)
- area of each particle (in pixels)
- bounding ellipse information
  - lengths of major and minor axes (in pixels)
  - angle (with respect to the x-axis) of major axis
- number of the frame/"slice" it belongs to
and the results of the border segmentation, which will give us 
coordinates of each point on the cell border as well as the number of
the slice it belongs to.

In [ ]:
# Read the "Analyze Particles" data exported from FIJI
data = pd.read_csv("/data/Results.csv")
# Read the border coordinates
borders = pd.read_csv("/data/border.txt", 
                      sep='\t', 
                      names=["x", "y", "slice", "val"], 
                      usecols=[0,1,2])
# Results use 1-indexed slice numbers, so subtract 1 to keep it 
# consistent with border indexing
data["Slice"] -= 1

### Compute and plot area distribution
Here we obtain the data in the "Area" column, which corresponds to the
area of each segmented focal adhesion, and plot it in a histogram.

In [ ]:
areas = data["Area"]
areas.plot.hist(bins=50, title="Area Distribution")

### Compute and plot aspect ratio distribution
Here we compute the aspect ratio, or ratio between the two dimensions of 
the focal adhesion, by dividing the major axis lengths by the minor axis lengths.
We then plot it in a histogram.

In [ ]:
major_axes = data["Major"]
minor_axes = data["Minor"]
aspect_ratios = major_axes / minor_axes
aspect_ratios.plot.hist(bins=50, title="Aspect Ratio Distribution")

### Compute angle with respect to border
Here we compute the angle of the major axis of each focal adhesion with
respect to the border. This analysis is somewhat more complicated than the
other statistics, and involves the following steps, repeated for each slice:
1. Get the border coordinates for the current slice; i.e. the entries
   in the border coordinates table whose slice number corresponds to
   the current one.
2. Similarly, get the focal adhesions in the current slice.
3. Build a $k$-d tree for each set of border coordinates, to allow for
   fast lookup of nearest neighbors.
4. For each focal adhesion, find the coordinates of the 10 border
   points nearest to its centroid, and compute a best-fit line for
   them. This will give us the slope $m_b$ and $y$-intercept of a line
   approximately tangent to the border.
   
   *Note*: The two nearest neighbors are not sufficient since the
   border is made up of pixels, so the two nearest neighbors often
   form vertical or horizontal lines, with slopes $\infty$ and $0$
   respectively.
5. For each focal adhesion, find the slope of its major axis; we have
   its angle $\theta_f$ so the slope is $m_f = \tan(\theta_f)$. In the
   code we must convert between degrees and radians since FIJI exports
   angles in degrees.
6. Compute the angle $\theta$ of each major axis with respect to the border
   using the following formula:
   $$\theta = \left|\tan^{-1}{\left(\frac{m_f-m_b}{1 + m_f \cdot
   m_b}\right)}\right|$$
   
The code is more complicated since it tries to operate on entire numpy
arrays whenever possible for efficiency, but it implements the above
algorithm.

In [ ]:
import warnings

# Produces an array of angles w.r.t the border for each focal adhesion in the given slice.
def get_angle_wrt_border_for_slice(n):
    # get border coordinate data for current slice
    current_border = borders.loc[borders["slice"] == n]
    # get focal adhesion data for current slice
    current_fas = data.loc[data["Slice"] == n].reset_index(drop=True)
    
    # get border points as an array of (x,y) tuples
    border_points = np.array(list(zip(current_border.x, current_border.y)))
    # get coordinates of focal adhesion centroids as an array of (x,y) tuples
    centroid_points = list(zip(current_fas.X, current_fas.Y))

    # build k-d tree of border points, and then query the 10 nearest neighbors for each
    # centroid point. This produces a 2-D array where rows are lists of neighbors for each
    # centroid point, and the values in those rows are the indices (w.r.t. the original
    # border_points array) of those neighbors
    indices = KDTree(border_points).query(centroid_points, k=10)[1]
    # Extract the x and y coordinates of the neighbors, using the indices. We now have two 2-D
    # arrays where rows are lists of neighbor x and y coordinates for each centroid point.
    neighbors_x = border_points.T[0].take(indices)
    neighbors_y = border_points.T[1].take(indices)

    # For every centroid point, find a best-fit line for its nearest neighbors and get the
    # slope of that line.
    border_slopes = np.array([linregress(neighbors_x[i], neighbors_y[i]).slope 
                              for i in range(0, len(indices))])
    # Get the slope of each focal adhesion's major axis.
    fa_axis_slopes = np.tan(np.radians(np.array(current_fas["Angle"])))
    
    # Compute angles with respect to the border using the two slopes, and return the resulting
    # array.
    angles_wrt_border = np.abs(np.degrees(np.arctan((fa_axis_slopes-border_slopes) /
                                                    (1 + fa_axis_slopes * border_slopes))))
    return angles_wrt_border

# Done to suppress warnings about dividing by zero; due to floating-point error we will
# sometimes do division by "zero" while computing the angle.
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    # Create a numpy universal function that we can apply to an array
    np_func = np.frompyfunc(get_angle_wrt_border_for_slice, 1, 1)
    # Apply the function to every slice.
    all_angles_wrt_border = np_func(np.arange(0, data["Slice"].max()+1))

# concatenate results, and insert them into the table.
all_angles_wrt_border = np.concatenate(all_angles_wrt_border)
data["Angle_wrt_Border"] = all_angles_wrt_border
# Plot the distribution in a histogram
data["Angle_wrt_Border"].plot.hist(bins=50, title="Distribution of Angle w.r.t. Border")